In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telecom/data2.csv
/kaggle/input/telecom/data1.csv


In [2]:
!pip install dask
!pip install psutil

In [3]:
import pandas as pd
import dask.dataframe as dd
import psutil
import time
import os

In [4]:
file_path = "/kaggle/input/telecom/data1.csv"

In [5]:
def memory_usage():
    process = psutil.Process(os.getpid())
    return round(process.memory_info().rss / (1024 * 1024), 2)  # بالميغا بايت

In [6]:
start_time = time.time()
mem_before = memory_usage()

chunksize = 1000000  # مليون صف في كل مرة
chunks = []
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunks.append(chunk.head(1))  # نقرأ صف واحد من كل chunk للتجريب
df_pandas = pd.concat(chunks)

mem_after = memory_usage()
time_pandas = time.time() - start_time

print(f"pandas + chunksize")
print(f"الوقت المستغرق: {time_pandas:.2f} ثانية")
print(f"الذاكرة المستخدمة: {mem_after - mem_before:.2f} MB")
print(f"عدد الصفوف المقروءة: {len(df_pandas)}")

pandas + chunksize
الوقت المستغرق: 242.54 ثانية
الذاكرة المستخدمة: 9781.83 MB
عدد الصفوف المقروءة: 161


In [7]:
start_time = time.time()
mem_before = memory_usage()

df_dask = dd.read_csv(file_path)
df_dask_head = df_dask.head(5)  # نقرأ فقط 5 أسطر

mem_after = memory_usage()
time_dask = time.time() - start_time

print(f"Dask")
print(f"الوقت المستغرق: {time_dask:.2f} ثانية")
print(f"الذاكرة المستخدمة: {mem_after - mem_before:.2f} MB")
print(df_dask_head.head())

Dask
الوقت المستغرق: 3.10 ثانية
الذاكرة المستخدمة: 108.23 MB
   GridID   TimeInterval  countrycode     smsin    smsout    callin   callout  \
0       1  1383260400000            0  0.081363       NaN       NaN       NaN   
1       1  1383260400000           39  0.141864  0.156787  0.160938  0.052275   
2       1  1383261000000            0  0.136588       NaN       NaN  0.027300   
3       1  1383261000000           33       NaN       NaN       NaN       NaN   
4       1  1383261000000           39  0.278452  0.119926  0.188777  0.133637   

    internet  
0        NaN  
1  11.028366  
2        NaN  
3   0.026137  
4  11.100963  


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [8]:
import gzip
import shutil
import time

# 🔹 المسار الأصلي للملف
file_path = "/kaggle/input/telecom/data1.csv"

# 🔹 مسار الملف المضغوط
compressed_file = "/kaggle/working/data1.csv.gz"

# ✅ 1. ضغط الملف (مرة واحدة فقط)
start_compress = time.time()
with open(file_path, 'rb') as f_in:
    with gzip.open(compressed_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
end_compress = time.time()

print(f"⏳ تم ضغط الملف بنجاح في {(end_compress - start_compress):.2f} ثانية.")
print(f"📦 حجم الملف بعد الضغط: {os.path.getsize(compressed_file) / (1024*1024):.2f} MB")

⏳ تم ضغط الملف بنجاح في 2169.65 ثانية.
📦 حجم الملف بعد الضغط: 2477.13 MB
